# Utilização de LLMs

In [1]:
import os
import warnings
import torch

from huggingface_hub import login

print(f"Device name: '{torch.cuda.get_device_name()}'")
print(f"Device properties: '{torch.cuda.get_device_properties(torch.cuda.current_device())}'")
print("Suporta bfloat16." if torch.cuda.is_bf16_supported() else "Não suporta bfloat16.")

Device name: 'NVIDIA GeForce RTX 2060 SUPER'
Device properties: '_CudaDeviceProperties(name='NVIDIA GeForce RTX 2060 SUPER', major=7, minor=5, total_memory=7974MB, multi_processor_count=34)'
Não suporta bfloat16.


In [2]:
warnings.filterwarnings('ignore')
login(token=os.environ["HUGGINGFACE_TOKEN"])

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
class LanguageModel:

    def __init__(self, tokenizer, model, device):
        self.tokenizer = tokenizer
        self.model = model
        self.device = device
        if self.tokenizer.pad_token_id is None:
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

    def tokenize(self, messages):
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        model_inputs = tokenizer([text], return_tensors="pt").to(self.device)
        return model_inputs

    def generate(self, messages):
        model_inputs = self.tokenize(messages)
        model_inputs['attention_mask'] = model_inputs['attention_mask'].to(model_inputs['input_ids'].device)
        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=512,
            do_sample=True,
            attention_mask=model_inputs['attention_mask'],
            pad_token_id=self.tokenizer.pad_token_id
        )
        generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]
        return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "microsoft/Phi-3-mini-4k-instruct"

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
%%time
llm = LanguageModel(tokenizer, model, device="cuda")

prompt = "Qual a capital do Brasil?"

messages = [
    {"role": "user", "content": "Olá"},
    {"role": "assistent", "content": "Tudo bem? Como posso ajudar?"},
    {"role": "user", "content": prompt},
]

llm.generate(messages)

You are not running the flash-attention implementation, expect numerical differences.


CPU times: user 2min 12s, sys: 842 ms, total: 2min 13s
Wall time: 2min 13s


'A capital do Brasil é Brasília. Foi fundada em 11 de maio de 1960, tornando-se a capital do país em 12 de outubro de 1960. A cidade foi projetada pelo arquiteto e urbanista brasileiro Lúcio Costa e pelo paisagista Roberto Burle Marx. A escolha da localização no meio da área central do país foi estratégica para promover o desenvolvimento do extremo oeste brasileiro.'